In [8]:

import os
import pandas as pd
import matplotlib.pyplot as plt
import rasterio
import numpy as np
from rasterio.transform import from_origin
from rasterio.windows import Window



### Cargamos la imagen con rasterio

Rasterio es la liberería que permitirá que las imágenes conserven los metadatos y que estén georeferenciadas aún después de procesarlas y recortarlas. 

In [2]:

# path  = '/content/drive/My Drive/doctorado_albert/conteo_pinguinos/'
# image_name = "orthomosaic_all_big.tif"

NUM_TILE = 62

if os.path.exists('/content/drive/My Drive/doctorado_albert/conteo_pinguinos/'):
    path  = '/content/drive/My Drive/doctorado_albert/conteo_pinguinos/'
else:
    path = 'G:\\.shortcut-targets-by-id\\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\\doctorado_albert\\conteo_pinguinos'

image_name = f"recortes/recorte_{NUM_TILE}.tif"

# Unir el path con el nombre de la imagen
tiff_file = os.path.join(path, image_name)

#! Verificamos la carga de la imagen (Solamente obtenemos información)
# Cargar la imagen con rasterio
with rasterio.open(tiff_file) as src:
    print(f"Imagen cargada desde: {tiff_file}")
    print("Dimensiones: ", (src.width, src.height))
    print("Transformación de coordenadas: ", src.transform)
    print("Sistema de coordenadas: ", src.crs)

Imagen cargada desde: G:\.shortcut-targets-by-id\1pYgV5EIk4-LapLNhlCwpQaDAzuqNffXG\doctorado_albert\conteo_pinguinos\recortes/recorte_62.tif
Dimensiones:  (10195, 11420)
Transformación de coordenadas:  | 0.00, 0.00,-59.24|
| 0.00,-0.00,-62.30|
| 0.00, 0.00, 1.00|
Sistema de coordenadas:  EPSG:4326


### Obtención de las coordenadas y de variables necesarias de la imagen

In [5]:
with rasterio.open(tiff_file) as src:
    # Obtener los metadatos
    transform = src.transform
    metadata = src.meta
    top_left = src.transform * (0, 0)
    top_right = src.transform * (src.width, 0)
    bottom_left = src.transform * (0, src.height)
    bottom_right = src.transform * (src.width, src.height)

    WIDTH, HEIGHT = src.width, src.height

   
    print("Metadatos de la imagen:")
    for key, value in metadata.items():
        print(f"{key}: {value}")

    # También puedes acceder a información específica si lo deseas
    print("\nInformación adicional:")
    print("Ancho de la imagen:", src.width)
    print("Altura de la imagen:", src.height)
    print("Sistema de coordenadas:", src.crs)  # Sistema de referencia de coordenadas
    print("Transformación:", src.transform)  # Transformación espacial


    print("Coordenadas de las esquinas de la imagen:")
    print("TOP LEFT:", top_left)
    print("Esquina superior derecha:", top_right)
    print("Esquina inferior izquierda:", bottom_left)
    print("BOTTOM RIGHT:", bottom_right)

  

Metadatos de la imagen:
driver: GTiff
dtype: uint8
nodata: None
width: 10195
height: 11420
count: 4
crs: EPSG:4326
transform: | 0.00, 0.00,-59.24|
| 0.00,-0.00,-62.30|
| 0.00, 0.00, 1.00|

Información adicional:
Ancho de la imagen: 10195
Altura de la imagen: 11420
Sistema de coordenadas: EPSG:4326
Transformación: | 0.00, 0.00,-59.24|
| 0.00,-0.00,-62.30|
| 0.00, 0.00, 1.00|
Coordenadas de las esquinas de la imagen:
TOP LEFT: (-59.236389267356785, -62.300974267474736)
Esquina superior derecha: (-59.231134009926784, -62.300974267474736)
Esquina inferior izquierda: (-59.236389267356785, -62.303714473634734)
BOTTOM RIGHT: (-59.231134009926784, -62.303714473634734)


In [9]:
#Directorio de salida
OUTPUT_DIR = os.path.join('maps', f'tiles_500x500_{NUM_TILE}')

#Definición del recorte
ROWS, COLS = 20, 20

# Dimensiones de cada recorte
tile_width = WIDTH // COLS
tile_height = HEIGHT // ROWS

# Crear un directorio para las imágenes recortadas
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Abrir el TIFF original con rasterio
with rasterio.open(tiff_file) as src:
    for i in range(ROWS):
        print("Fila nº ", i)
        for j in range(COLS):
            left = j * tile_width
            upper = i * tile_height
            
            # Definir ventana de recorte
            window = Window(left, upper, tile_width, tile_height)
            
            # Leer el recorte
            cropped_image = src.read(window=window)
            
            # Crear los metadatos del recorte
            cropped_meta = src.meta.copy()
            cropped_meta.update({
                "height": tile_height,
                "width": tile_width,
                "transform": rasterio.windows.transform(window, src.transform)
            })
            
            # Guardar la imagen recortada con georreferenciación
            output_path = f"{OUTPUT_DIR}/subrecorte_{i * COLS + j + 1}.tiff"
            with rasterio.open(output_path, 'w', **cropped_meta) as dst:
                dst.write(cropped_image)

print("Recortes guardados en el directorio:", OUTPUT_DIR)

Fila nº  0
Fila nº  1
Fila nº  2
Fila nº  3
Fila nº  4
Fila nº  5
Fila nº  6
Fila nº  7
Fila nº  8
Fila nº  9
Fila nº  10
Fila nº  11
Fila nº  12
Fila nº  13
Fila nº  14
Fila nº  15
Fila nº  16
Fila nº  17
Fila nº  18
Fila nº  19
Recortes guardados en el directorio: maps\tiles_500x500_62


### Comprobamos que los recortes están efectivamente georeferenciados

In [10]:
OUTPUT_DIR = os.path.join('maps', f'tiles_500x500_{NUM_TILE}')
sample_file = os.path.join(OUTPUT_DIR, "test_tile.tiff")

with rasterio.open(sample_file) as src:
    transform = src.transform
    metadata = src.meta
    top_left = src.transform * (0, 0)
    top_right = src.transform * (src.width, 0)
    bottom_left = src.transform * (0, src.height)
    bottom_right = src.transform * (src.width, src.height)

    WIDTH, HEIGHT = src.width, src.height

   
    print("Metadatos de la imagen:")
    for key, value in metadata.items():
        print(f"{key}: {value}")

    # También puedes acceder a información específica si lo deseas
    print("\nInformación adicional:")
    print("Ancho de la imagen:", src.width)
    print("Altura de la imagen:", src.height)
    print("Sistema de coordenadas:", src.crs)  # Sistema de referencia de coordenadas
    print("Transformación:", src.transform)  # Transformación espacial


    print("Coordenadas de las esquinas de la imagen:")
    print("TOP LEFT:", top_left)
    print("Esquina superior derecha:", top_right)
    print("Esquina inferior izquierda:", bottom_left)
    print("BOTTOM RIGHT:", bottom_right)

Metadatos de la imagen:
driver: GTiff
dtype: uint8
nodata: None
width: 509
height: 571
count: 4
crs: EPSG:4326
transform: | 0.00, 0.00,-59.23|
| 0.00,-0.00,-62.30|
| 0.00, 0.00, 1.00|

Información adicional:
Ancho de la imagen: 509
Altura de la imagen: 571
Sistema de coordenadas: EPSG:4326
Transformación: | 0.00, 0.00,-59.23|
| 0.00,-0.00,-62.30|
| 0.00, 0.00, 1.00|
Coordenadas de las esquinas de la imagen:
TOP LEFT: (-59.231666494568785, -62.30124828809073)
Esquina superior derecha: (-59.231404118302784, -62.30124828809073)
Esquina inferior izquierda: (-59.231666494568785, -62.301385298398735)
BOTTOM RIGHT: (-59.231404118302784, -62.301385298398735)
